In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2 as cv

In [3]:
path = "archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"

train = os.listdir(path + "/train")
key_value = dict()

for t in range(len(train)):
    key_value[train[t]] = os.listdir(path+'/train/'+train[t])

In [4]:
df = []
for i in range(len(train)):
    for j in range(len(key_value[train[i]])):
        df.append([path+'/train/'+train[i]+'/'+key_value[train[i]][j],train[i]])

In [5]:
test = pd.DataFrame(df,columns=['image','label'])
test

,image,label
0,archive/New Plant Diseases Dataset(Augmented)/...,Apple___Apple_scab
1,archive/New Plant Diseases Dataset(Augmented)/...,Apple___Apple_scab
2,archive/New Plant Diseases Dataset(Augmented)/...,Apple___Apple_scab
3,archive/New Plant Diseases Dataset(Augmented)/...,Apple___Apple_scab
4,archive/New Plant Diseases Dataset(Augmented)/...,Apple___Apple_scab
...,...,...
70290,archive/New Plant Diseases Dataset(Augmented)/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
70291,archive/New Plant Diseases Dataset(Augmented)/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
70292,archive/New Plant Diseases Dataset(Augmented)/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
70293,archive/New Plant Diseases Dataset(Augmented)/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [6]:
x = []
for i in test['image']:
    img = cv.imread(i)
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(64,64))
    x.append(img)

In [7]:
x = np.array(x).reshape(70295, 64, 64,1)

In [8]:
np.save('image_array',x)

In [9]:
y = test['label']   
le = LabelEncoder()
y_label = le.fit_transform(y)

In [10]:
train_images,test_images,train_labels,test_labels = train_test_split(x,y_label,test_size=0.2,random_state=42)

In [11]:
train_images = train_images/255
test_images = test_images/255

In [12]:
y_test = pd.DataFrame(train_labels)

In [13]:
model = Sequential()
model.add(Conv2D(128,(5,5),activation='relu',input_shape=(64,64,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(256,(3,3),activation='relu',input_shape=(64,64,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(256,(3,3),activation='relu',input_shape=(64,64,1)))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(38,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
model.fit(train_images,y_test,epochs=1,validation_data = (test_images,test_labels))

1758/1758 [==============================] - 15609s 9s/step - loss: 1.5520 - accuracy: 0.5406 - val_loss: 1.0768 - val_accuracy: 0.6563


In [15]:
model.save('predictionmodel.h5')

In [16]:
from tensorflow import keras

In [19]:
loaded_model = keras.models.load_model('predictionmodel.h5')

In [20]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 128)       3328      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       590080    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 256)        0